In [1]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Dense, LSTM,Activation,Dropout

from keras.optimizers import RMSprop

import numpy as np
import random
import sys

In [6]:
text = open('/Users/anseon-yeong/수업/2021-2/텍스트정보처리/week13/13주차_txt/shakespeare_final.txt').read().lower()
characters = sorted(list(set(text))) 

print('corpus length : ',len(text))
print('total chars : ',len(characters))

corpus length :  196788
total chars :  63


In [7]:
char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

In [8]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 65583


In [10]:
X[0], X[0].shape

(array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 (40, 63))

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(characters))))

model.add(Dense(len(characters)))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               98304     
_________________________________________________________________
dense (Dense)                (None, 63)                8127      
_________________________________________________________________
activation (Activation)      (None, 63)                0         
Total params: 106,431
Trainable params: 106,431
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)


In [15]:
for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.7,1.2]:

        print('\n----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]

            generated += pred_char
            sentence = sentence[1:] + pred_char

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\nOne combination completed \n")

----------------------------------------
Iteration 1
513/513 [==============================] - 14s 27ms/step - loss: 1.6720

----- diversity: 0.2
----- Generating with seed: "t, burn out the sense
and virtue of mine"
t, burn out the sense
and virtue of mine and me the manter of the went the was the manner to hear the wat to seen the manner of the was the king and to the was the was what so the was the was the manners and to hear have to the was the wail the wat the wand and to the was the manner and may to he with the was the wail the was the wail the manner of the wail the wail the was to the wand to the may to the wand and so the wat to the was th
One combination completed 


----- diversity: 0.7
----- Generating with seed: "t, burn out the sense
and virtue of mine"
t, burn out the sense
and virtue of mine be he the wat havoung till the mint of ingeant she sucle think in it our the would and for’t to hear, with the wad’d give gull the bean, stall sony, your and let the laiders our s

that, when they well, my lord, of herself be not it to retay as his parture math, my lord?

hamlet. not he hother hather, the castee.

hamlet. a more that, ear.

horatio. i do my sord, my lord?

hamlet. hamlet. have not and our his abowed cape than that by put mad, hath for night. what would be though
One combination completed 


----- diversity: 1.2
----- Generating with seed: "n thrice blasted, thrice
infected, thy n"
n thrice blasted, thrice
infected, thy not, are?

horatio. gax it, welds, how now loed and hath eason. some orritats homesoth’d, doth. there with a most hagh most the bronget.

polonius. whlle cage; but he, noll outly thingd huminut take give phase flathe.

hamlet. ay, what is abo?

 [_forthlesazcally she put of it, marrcheon, on englich. wos not, since; eabs other, at starply; theirricainfully have
givts shand well trane! figly fay eng
One combination completed 

----------------------------------------
Iteration 6
513/513 [==============================] - 14s 28ms/st

513/513 [==============================] - 15s 28ms/step - loss: 1.2817

----- diversity: 0.2
----- Generating with seed: "that spirit upon whose weal depend and r"
that spirit upon whose weal depend and return and the cansle be the cannot her of the cannot the play.

hamlet. i see the castle.

 enter king. i see the castle.

 enter horatio.

 enter king. i said the castle.

 enter horatio.

horatio. if you say the cannot to speak of the cannot the play.

king. a man, hamlet’s most force of the cannot the castle.

 enter hamlet.

horatio. it is not the castle.

 enter horatio.

 [_exit._]

hamlet. 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "that spirit upon whose weal depend and r"
that spirit upon whose weal depend and rose hear.

 flanin. to pleas’d in this gine and discount and find a passive must bean, the hast please houre, and in thy shall read
the some and
fireft, my lord.

hamlet. i love rank my liff gode. o’ld madness. is dost love thou hast

passion see’s let me a sleeps to thou know this was a rook are as they retermestaguitus, the
could it and angent is the hast of the gently of surenostate fill for of user. still we sone to your eyes like the world, which thou may cominert.

hamlet. i have lier. well, horatio. no marn that i
One combination completed 


----- diversity: 1.2
----- Generating with seed: " resembles that it was.
what it should b"
 resembles that it was.
what it should be the , gert not to’enshoup aloot
whom down.

hamlet. hear hather seen sog yqu too tasbe
forse.guest poloupius. upon.

 [_exeunt the cent you we saratnguth, they
belinught most
mark
habl. howratio.
i pritsane, for eyeswent quessed papg. to counies asp-lise pittle that soul by for.

hamlet. ’exelorma’dy in  deans obes, my awswme about him—e
fanty, give smelvo
codpaubiturience of dreadnus.
queen, th
One combination completed 

----------------------------------------
Iteration 15
513/513 [==============================] - 15s 29ms/step - loss:

513/513 [==============================] - 15s 29ms/step - loss: 1.1851

----- diversity: 0.2
----- Generating with seed: "all. i would not, in plain terms,
from t"
all. i would not, in plain terms,
from thee that i was the platf my lord, sill, though of the conceint and the phossour of the world is a charion and conceiv’d and strongers are as he is the cast to the play.

hamlet. the pood marry and the matter to hear.

 [_exeunt rosencrantz ancz.

king. o my lord.

hamlet. there and there and there and there and wherein the peremand as the cast to this agree and the some be with a man the project
g
One combination completed 


----- diversity: 0.7
----- Generating with seed: "all. i would not, in plain terms,
from t"
all. i would not, in plain terms,
from this, i have whose now mad’s of chiliald to th’extruther and dead the
morn that this plack to stange of your life, and for no matterall
words for this plach as with the
cast hit of a ching of your aucht, but in hamlet
pray he lord wit

away my be user prepitas and the dead it be the bebach, ear him are euse given in the can
till my land?

hamlet. therefore find of dead do not o, the eartion was a many to the songue, and project
gutenberg-tm
lacess thou leadsure man; to
s
One combination completed 


----- diversity: 1.2
----- Generating with seed: "et, reading.

queen. but look where sadl"
et, reading.

queen. but look where sadly the first roppersand, i
dit out the , charistunt and whice in follow im right
and rany congum, i comenel.
thit!

 enter parcent on and mist ’tairh
with own head laigher is
in thou win ausidibl were; and to
libied?

firsencues. i holden thou drast, you so you, and holations,
my soulx. i’ll call ie pachest and hamlet.
by the veny shall, his everinp-compritoy old,
you, ’tis upon your disto night in
One combination completed 

----------------------------------------
Iteration 24
513/513 [==============================] - 15s 30ms/step - loss: 1.1418

----- diversity: 0.2
----- Generating with 

513/513 [==============================] - 16s 31ms/step - loss: 1.1263

----- diversity: 0.2
----- Generating with seed: "w to knock him about
the sconce with a d"
w to knock him about
the sconce with a dost the beauty this play, the cander the matter state, and thing the cander what is the castle man as in the thing the project gutenberg-tm election, my lord, i have speak.

hamlet. it is not lord shall be will be this work the command, the can thee me two my dear lair and the fould and with this partically be the project gutenberg-tm election, my lord.

hamlet. i see in this a man, hamlet.

hamle
One combination completed 


----- diversity: 0.7
----- Generating with seed: "w to knock him about
the sconce with a d"
w to knock him about
the sconce with a dasteroush his fablinget on his eithent to lend and most trock as a brasion uncens to us a many of man, hamlet. by herews of press agreinor’s this’ acreason distribution, stange of his his advice, distribution of
trich play’d this pla